In [1]:
%load_ext cython
%load_ext line_profiler
import re

import line_profiler
from collections import Counter

In [2]:
from Cython.Compiler import Options 

defaults = Options.get_directive_defaults()
defaults['linetrace'] = True
defaults['binding'] = True

In [3]:
def my_poly(a,b):
    return 10.5 * a + 3 * (b**2)

In [ ]:
%%cython 
def my_polyx(double a, double b):
    return 10.5 * a + 3 * (b**2)

In [ ]:
%timeit my_poly(10, 2)
%timeit my_polyx(10, 2)  #cython 3x faster

353 ns ± 6.36 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
import pandas as pd
df = pd.read_csv('data/pwAnalysis.csv',usecols=['A'])

In [ ]:
s = df[:1000].A.str.cat()
print(s[:200])

In [ ]:
def mostCommonStrings(charLen, s):
    cnt = {}
    t = len(s) - (charLen - 1)
    for i in range(t):
        target = s[i:i+charLen]
        
        # let's only look at alpha strings
#         allAlpha = True
#         for j in range(charLen):
#             if not target[j].isalpha():
#                 allAlpha = False
#                 break
        
#         if allAlpha:
        cnt[target] = s.count(target)
    return cnt

In [ ]:
def getCount(target,s):
    return len(re.findall(target,s))

In [ ]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
def mostCommonStringsX(int charLen, str s):
    cdef dict cnt = {}
    cdef int i = 0
    cdef int t = 0
    cdef int numFound = 0;
    cdef str target = ''
    t = len(s) - (charLen - 1)
    for i in range(t):
        target = s[i:i+charLen]
#         numFound = s.count(target)
        cnt[target] = getCount(target,s)
#     s.count(target)
    return cnt

In [ ]:
%timeit mostCommonStrings(4,s)
%timeit mostCommonStringsX(4,s)

In [ ]:
%%time
cnt = Counter(mostCommonStringsX(4,s))
print(cnt.most_common(5))

## profile mostCommonStringsX

In [ ]:
#Print profiling statistics using the `line_profiler` API
profile = line_profiler.LineProfiler(mostCommonStringsX)
profile.runcall(mostCommonStringsX,4,s)
profile.print_stats()